<a href="https://colab.research.google.com/github/shiffa-04/IMDB_SentimentAnalysis_NLP/blob/main/IMDB_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
import pandas as pd
import re
import string
import nltk
import gensim
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

np.random.seed(42)

In [3]:
df = pd.read_csv("IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
classes = df['sentiment'].value_counts()
classes

,count
sentiment,
positive,25000
negative,25000


In [5]:
df.shape

(50000, 2)

In [6]:
df.duplicated().sum()

418

In [7]:
df.drop_duplicates(inplace = True)
df.duplicated().sum()

0

In [8]:
df['review'] = df['review'].str.lower()
df.head(2)

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive


In [9]:
def remove_html_tags(review):
  clean = re.compile('<.*?>')
  return re.sub(clean, '', review)

df['review'] = df['review'].apply(remove_html_tags)
df['review'][1]

'a wonderful little production. the filming technique is very unassuming- very old-time-bbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. the actors are extremely well chosen- michael sheen not only "has got all the polari" but he has all the voices down pat too! you can truly see the seamless editing guided by the references to williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. a masterful production about one of the great master\'s of comedy and his life. the realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. it plays on our knowledge and our senses, particularly with the scenes concerning orton and halliwell and the sets (particularly of their flat with halliwell\'s murals decorating every surface) are terribly well done.'

In [10]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
# lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

In [12]:
reviews = df['review']

In [13]:
print(reviews)

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. the filming tec...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 49582, dtype: object


In [14]:
def text_pre_processing(text):
  # corpus =[]
  cleaned_message = re.sub('[^a-zA-Z]', ' ', text)

  #removing stopwords:
  words = cleaned_message.split()
  filtered_words = [word for word in words if word not in stopwords.words('english')]

  processed_words = [ps.stem(word) for word in filtered_words]
  #Lemmatization:
  # lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

  processed_review = ' '.join(processed_words)

  return processed_review


In [15]:
df['cleaned_review'] = reviews.apply(text_pre_processing)

In [16]:
X = df['cleaned_review']
print(X)

0        one review mention watch oz episod hook right ...
1        wonder littl product film techniqu unassum old...
2        thought wonder way spend time hot summer weeke...
3        basic famili littl boy jake think zombi closet...
4        petter mattei love time money visual stun film...
                               ...                        
49995    thought movi right good job creativ origin fir...
49996    bad plot bad dialogu bad act idiot direct anno...
49997    cathol taught parochi elementari school nun ta...
49998    go disagre previou comment side maltin one sec...
49999    one expect star trek movi high art fan expect ...
Name: cleaned_review, Length: 49582, dtype: object


In [17]:
text_list = X.tolist()

In [19]:
all_processed_words = ' '.join(X).split()

words_in_corpus = len(all_processed_words)
vocabulary = len(set(all_processed_words))

print(f'Total words in Corpus: {words_in_corpus}')
print(f'Total words in vocabulary: {vocabulary}')

Total words in Corpus: 5862623
Total words in vocabulary: 70962


In [18]:
le = LabelEncoder()
y = le.fit_transform(df['sentiment'])
print(y)

[1 1 1 ... 0 0 0]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2, random_state=42)

**USING BAG OF WORDS**

In [ ]:
vectorizer_bow = CountVectorizer(ngram_range=(1, 2), max_features = 5000)
X_train_bow = vectorizer_bow.fit_transform(X_train)
X_test_bow = vectorizer_bow.transform(X_test)

# print(X_train_bow)
# print(X_test_bow)

In [ ]:
vocabulary = vectorizer_bow.get_feature_names_out()
print(f"Vocabulary size: {len(vocabulary)}")

Vocabulary size: 5000


In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_bow, y_train)

# Make predictions
y_pred = rf_model.predict(X_test_bow)

In [ ]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8389633961883635
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.83      0.84      4939
           1       0.84      0.85      0.84      4978

    accuracy                           0.84      9917
   macro avg       0.84      0.84      0.84      9917
weighted avg       0.84      0.84      0.84      9917



**USING TF-IDF**


In [ ]:
# TF-IDF:
vectorizer_tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features = 5000)
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

In [ ]:
vocabulary = vectorizer_tfidf.get_feature_names_out()
print(f"Vocabulary size: {len(vocabulary)}")

Vocabulary size: 5000


In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = rf_model.predict(X_test_tfidf)

In [ ]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8425935262680246
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.84      0.84      4939
           1       0.84      0.85      0.84      4978

    accuracy                           0.84      9917
   macro avg       0.84      0.84      0.84      9917
weighted avg       0.84      0.84      0.84      9917



**USING WORD2VEC**

In [ ]:
import gensim

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))


In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
model.build_vocab(story)

In [ ]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(41729468, 54449140)

In [ ]:
len(model.wv.index_to_key)

61844

In [ ]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [ ]:
document_vector(df['review'].values[0])

array([ 5.85180596e-02,  1.80571839e-01,  1.95474699e-01, -1.96748182e-01,
       -2.25829437e-01,  5.46702743e-01,  6.27389178e-02,  1.28377276e-02,
        3.37910175e-01, -2.47808173e-01, -9.88389086e-03, -6.90566778e-01,
       -5.54880023e-01, -2.17485175e-01, -4.14704382e-01,  1.19858176e-01,
       -3.41899335e-01, -2.97005754e-02, -8.50120783e-01,  5.06304562e-01,
       -3.77325207e-01, -2.39909932e-01, -2.12296814e-01,  3.05089474e-01,
       -4.48380142e-01, -7.22336888e-01,  1.45542309e-01, -2.40610734e-01,
       -2.88048387e-01, -1.40392818e-02, -3.29195112e-02,  1.79478317e-01,
        7.50608295e-02,  3.57027322e-01,  1.04424417e+00,  1.60519943e-01,
       -2.46176615e-01,  2.99966097e-01,  5.21757901e-01, -3.31771284e-01,
       -7.44098623e-04, -8.83440256e-01, -3.30787227e-02, -5.99632561e-02,
        5.38305342e-01,  3.57004851e-01, -2.69443929e-01,  2.76537210e-01,
        6.65648043e-01,  7.37287164e-01, -1.20332807e-01, -4.54499513e-01,
        2.09775552e-01,  

In [ ]:
from tqdm import tqdm

In [ ]:
X1 = []
for doc in tqdm(df['review'].values):
    X1.append(document_vector(doc))

100%|██████████| 49582/49582 [3:55:43<00:00,  3.51it/s]


In [ ]:
X1 = np.array(X1)

In [ ]:
X1[0]

array([ 5.85180596e-02,  1.80571839e-01,  1.95474699e-01, -1.96748182e-01,
       -2.25829437e-01,  5.46702743e-01,  6.27389178e-02,  1.28377276e-02,
        3.37910175e-01, -2.47808173e-01, -9.88389086e-03, -6.90566778e-01,
       -5.54880023e-01, -2.17485175e-01, -4.14704382e-01,  1.19858176e-01,
       -3.41899335e-01, -2.97005754e-02, -8.50120783e-01,  5.06304562e-01,
       -3.77325207e-01, -2.39909932e-01, -2.12296814e-01,  3.05089474e-01,
       -4.48380142e-01, -7.22336888e-01,  1.45542309e-01, -2.40610734e-01,
       -2.88048387e-01, -1.40392818e-02, -3.29195112e-02,  1.79478317e-01,
        7.50608295e-02,  3.57027322e-01,  1.04424417e+00,  1.60519943e-01,
       -2.46176615e-01,  2.99966097e-01,  5.21757901e-01, -3.31771284e-01,
       -7.44098623e-04, -8.83440256e-01, -3.30787227e-02, -5.99632561e-02,
        5.38305342e-01,  3.57004851e-01, -2.69443929e-01,  2.76537210e-01,
        6.65648043e-01,  7.37287164e-01, -1.20332807e-01, -4.54499513e-01,
        2.09775552e-01,  

In [ ]:
from sklearn.model_selection import train_test_split
X1_train,X1_test,y1_train,y1_test = train_test_split(X1, y, test_size=0.2, random_state=1)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X1_train,y1_train)
y_pred = rf.predict(X1_test)
accuracy_score(y1_test,y_pred)

0.7923767268327115

In [ ]:
print("Classification Report:\n", classification_report(y1_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.77      0.79      5033
           1       0.78      0.81      0.79      4884

    accuracy                           0.79      9917
   macro avg       0.79      0.79      0.79      9917
weighted avg       0.79      0.79      0.79      9917



**SENTENCE EMBEDDING**

In [ ]:
pip install sentence-transformers

In [20]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = text_list
sentence_embeddings = model.encode(sentences)

print(sentence_embeddings)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.00413832 -0.06671323 -0.06629918 ... -0.04595426 -0.00981964
  -0.00511605]
 [-0.05986906 -0.02205939  0.05380616 ... -0.03434895 -0.01630318
   0.00556395]
 [-0.0430442  -0.08457743  0.01312509 ... -0.02342531 -0.09143322
  -0.04141012]
 ...
 [ 0.0470698  -0.03338439 -0.04066566 ... -0.00617372  0.04248716
  -0.01971653]
 [ 0.00219575 -0.0513603  -0.00789999 ...  0.01684531  0.00817453
  -0.05542235]
 [-0.08422829 -0.12204494  0.07859126 ... -0.00097408 -0.06501717
  -0.03938875]]


In [21]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(sentence_embeddings, y, test_size=0.2, random_state=1)

In [22]:
rf = RandomForestClassifier()
rf.fit(X_train_s, y_train_s)
y_pred_s = rf.predict(X_test_s)
accuracy_score(y_test_s, y_pred_s)

0.7370172431178784

In [24]:
print("Classification Report:\n", classification_report(y_test_s, y_pred_s))

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.71      0.73      5033
           1       0.72      0.77      0.74      4884

    accuracy                           0.74      9917
   macro avg       0.74      0.74      0.74      9917
weighted avg       0.74      0.74      0.74      9917



In [32]:
# Train and evaluate the model using XGBoost
xg = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xg.fit(X_train_s, y_train_s)
y_pred_xg = classifier.predict(X_test_s)
accuracy_score(y_test_s, y_pred_xg)

# Print classification report
print("Classification Report:\n", classification_report(y_test_s, y_pred_xg))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:25:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.75      0.76      5033
           1       0.75      0.78      0.76      4884

    accuracy                           0.76      9917
   macro avg       0.76      0.76      0.76      9917
weighted avg       0.76      0.76      0.76      9917

